# Q7: Color Quantization

# # map every color in an rgb image to cloest color in your color wheel.

# you can check the results in my MidtermQ7 folder.

To find the distance of two color. 
Map the cloest color in my color wheel to my image color.
For each pixel, applying this rule until it done.

In [1]:
from colorsys import rgb_to_hsv
from PIL import Image, ImageDraw, ImageOps
colorwheels = dict((
((196, 2, 51), "C1"),
((255, 165, 0), "C2"),
((255, 205, 0), "C3"),
((0, 128, 0), "C4"),
((0, 0, 255), "C5"),
((127, 0, 255), "C6"),
((210, 255, 162), "C7"),
((242, 129, 255), "C8"),))


dir = '/Users/yuelingqin/Desktop/infor/4300image/'
def to_hsv(color): 
    #print(color) rgb
    """ converts color tuples to floats and then to hsv """
    #print(*[x/255.0 for x in color])
    #return rgb_to_hsv(*[x/255.0 for x in color]) #rgb_to_hsv wants floats!
    return (x/255.0 for x in color)

def color_dist(c1, c2):
    """ returns the squared euklidian distance between two color vectors in hsv space """
    #print c1 rgb
    return sum((a-b)**2 for a,b in zip(to_hsv(c1),to_hsv(c2)) )

  
def min_color_diff(img,colorwheels):
    """ returns the `(distance, color_name)` with the minimal distance to `colors`"""
    (width, height) = img.size
    #print colorwheels
    for i in xrange(width):
        for j in xrange(height):
            r,g,b = img.getpixel((i,j))
            color_to_match = (r,g,b)
            for x,y in colorwheels.items():
                rbg,colorwheel = min( # overal best is the best match to any color:
                    (color_dist(color_to_match, x), colorwheels[x]) for x in colorwheels)
                #print (x)
                if (colorwheel == y):
                    img.putpixel((i,j), x)


    return img



# handle color image

In [2]:
    
img = Image.open(dir + "23.jpg")
img = img.convert('RGB')
im = min_color_diff(img, colorwheels)
im.show()
im.save('MyImage.png')

# handle grayscale image

In [3]:

image_2 = Image.open(dir + "TEST.png")
image_2 = image_2.convert('RGB')
grayresult = min_color_diff(image_2, colorwheels)
grayresult.save("gray.png")


In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from skimage import io
from PIL import Image, ImageDraw, ImageOps
from skimage import segmentation as seg

dir = '/Users/yuelingqin/Desktop/infor/4300image/'

image_1 = io.imread(dir + '33.jpg', as_grey=True)
image_2 = io.imread(dir + 'greyscale.jpg')
image_3 = io.imread(dir + 'drawn.jpg')
image_4 = io.imread(dir + '44.jpeg')
image_5 = io.imread(dir + '23.jpg')


image_1_sc = io.imread(dir + '33.jpg', )
image_test = io.imread('/Users/yuelingqin/Desktop/' +'MyImage.png', )


##According to color freq, we can update my color wheel.

find color freq of my image.
based on high freq to update my color wheel.
apply first funtion to draw my new image by using new color wheel.

In [5]:
# find the color with highest frequency 
def freq_color(image, labels):
    out = np.zeros_like(image)
    colors = []
    for label in np.unique(labels):
        indices = np.nonzero(labels == label)
        rgbArray = image[indices]
        r = rgbArray[..., 0]
        g = rgbArray[..., 1]
        b = rgbArray[..., 2]
        fr = high_freq(r)
        fg = high_freq(g)
        fb = high_freq(b)
        out[indices] = (fr,fg,fb)
        c = (fr,fg,fb)
        colors.append(np.int_(c))
    return out, colors


# helper method, find number appears most frequently in array
def high_freq(arr):
    y = np.bincount(arr)
    ii = np.nonzero(y)[0]
    map = zip(ii,y[ii]) 
    n = map[0][1]
    v = map[0][0]
    for i in range(0, len(map)):
        if map[i][1]>n:
            n=map[i][1]
            v=map[i][0]
    return v



# draw the output 
def draw_FreqColor(image, n): 
    downsample = image[::6,::6,:]
    labels = seg.slic(downsample, n_segments=n+1, compactness=15)
    out, colors = freq_color(downsample, labels)
#     colors = ["#5A31B4","#8A64E0","#888800","#008800","#008888","#000088","#440088","#880088"]
    labels_image = out
    # draw the color 
    pal = Image.new('RGB', (20*n, 20))
    draw = ImageDraw.Draw(pal)
    posx = 0 
    colorwheel = {}
    
    for i in range(0,n):
        draw.rectangle([posx, 0, posx+20, 20], fill=(colors[i][0],colors[i][1],colors[i][2]))
        colorwheel[(colors[i][0], colors[i][1], colors[i][2])] = "C"+str(i+1)
        posx = posx + 20

    return colorwheel;

# apply first code into the image.

In [10]:
colorwheel = draw_FreqColor(image_5,8);
im2 = min_color_diff(image_5,colorwheel)
im2.show()
im2.save('freqimage.png')
#{(255, 205, 0): 'YELLOW', (242, 129, 255): 'PINK', (0, 0, 255): 'BLUE', (255, 165, 0): 'ORANGE', (196, 2, 51): 'RED', (127, 0, 255): 'VIOLET', (210, 255, 162): 'LIGHTGREEN', (0, 128, 0): 'GREEN'}

In [12]:
image2 = Image.open(dir + "TEST.png")
image2 = image2.convert('RGB')
grayresult = min_color_diff(image2, colorwheel) 
grayresult.save("gray.png") #image saved in my MidtermQ7 folder, you can check the reuslt.